In [121]:
from hatchet import *
import json
from state import State
from preprocess import PreProcess
from filter import Filter
import numpy as np
import utils
import random
import os

In [181]:
# dirname = '/Users/jarus/ucd/Research/Visualisation/projects/CallFlow/.callflow'
dirname = '/home/vidi/Work/llnl/CallFlow/.callflow'
dataset = 'calc-pi'

In [182]:
pd.options.display.max_rows = 10
pd.options.display.float_format = '{:,.2f}'.format

# Read entire dataset

In [183]:
def replace_str_with_Node(df, graph):
        mapper = {}
        def dfs_recurse(root):
            for node in root.children: 
                mapper[node.callpath[-1]] = Node(node.nid, node.callpath, None)
                dfs_recurse(node)
        for root in graph.roots:
            mapper[root.callpath[-1]] = Node(root.nid, root.callpath, None)
            dfs_recurse(root)
        df['node'] = df['node'].apply(lambda node: mapper[node] if node in mapper else '')
        return df

In [184]:
def write_gf(state, state_name, format_of_df, write_graph=True):
    print('writing file for {0} format'.format(format_of_df))

    if write_graph:
        # dump the entire_graph as literal
        graph_literal = state.graph.to_literal(graph=state.graph, dataframe=state.df)
        graph_filepath = dirname + '/' + state_name + '/' + format_of_df + '_graph.json'
        print('File path: {0}'.format(graph_filepath))
        with open(graph_filepath, 'w') as graphFile:
            json.dump(graph_literal, graphFile)

    # dump the filtered dataframe to csv.
    df_filepath = dirname + '/' + state_name + '/' + format_of_df + '_df.csv'
    state.df.to_csv(df_filepath)

In [185]:
def read_gf(name):
    state = State()
    df_filepath = dirname + '/' + name +  '/filter_df.csv'
    entire_df_filepath = dirname + '/' + name + '/entire_df.csv'
    graph_filepath = dirname + '/' + name + '/filter_graph.json'
    entire_graph_filepath = dirname + '/' + name + '/entire_graph.json'   
    group_df_filepath = dirname + '/' + name + '/group_df.csv'

    with open(graph_filepath, 'r') as graphFile:
        data = json.load(graphFile)

    state.gf = GraphFrame()
    state.gf.from_literal(data)

    with open(entire_graph_filepath, 'r') as entire_graphFile:
        entire_data = json.load(entire_graphFile)
            
    state.entire_gf = GraphFrame()
    state.entire_gf.from_literal(entire_data)

    state.df = pd.read_csv(df_filepath)
    state.entire_df = pd.read_csv(entire_df_filepath)

    state.graph = state.gf.graph
    state.entire_graph = state.entire_gf.graph

    # replace df['node'] from str to the Node object.
    state.df = replace_str_with_Node(state.df, state.graph)
    state.entire_df = replace_str_with_Node(state.entire_df, state.entire_graph)
    
    state.group_df = pd.read_csv(group_df_filepath)

    return state

In [186]:
def ffilter(state, filterBy, filterPerc):
        filter_obj = Filter(state, filterBy, filterPerc)

        state.gf = filter_obj.gf
        state.df = filter_obj.df
        state.graph = filter_obj.graph

        return state

In [187]:
state = read_gf(dataset)

In [162]:
print(state.df)

                      node  rank                  node.1  rank.1  time (inc)  \
0           <program root>     0          <program root>       0  246,611.89   
1           <program root>     1          <program root>       1  902,982.46   
2           <program root>     3          <program root>       3  609,100.32   
3                     main     0                    main       0  823,426.11   
4                     main     1                    main       1  670,229.85   
..                     ...   ...                     ...     ...         ...   
18       294:MPID_Finalize     3       294:MPID_Finalize       3  589,598.43   
19  162:MPIDI_CH3_Finalize     0  162:MPIDI_CH3_Finalize       0  319,197.57   
20  162:MPIDI_CH3_Finalize     1  162:MPIDI_CH3_Finalize       1  819,456.53   
21  162:MPIDI_CH3_Finalize     2  162:MPIDI_CH3_Finalize       2  212,572.48   
22  162:MPIDI_CH3_Finalize     3  162:MPIDI_CH3_Finalize       3  795,559.32   

    time  nid  rank.1.1                

# Half-calc-pi dataset

In [195]:
def setvalues(state):
    random = pd.Series(np.random.rand(state.df.shape[0]))
    print(random)
    state.df['time (inc)'] = state.df['time (inc)']*random
    state.df['time'] = state.df['time']*random
    state.entire_df['time (inc)'] = state.entire_df['time (inc)']*random
    state.entire_df['time'] = state.entire_df['time']*random
    state.group_df['time (inc)'] = state.group_df['time (inc)']*random
    return state

In [196]:
def create_dot_callflow_folder(dataset):
            dataset_dir = dirname + '/' + dataset
            if not os.path.exists(dataset_dir):
                utils.debug('Creating .callflow directory for dataset : {0}'.format(dataset))
                os.makedirs(dataset_dir)
    
            files = ["entire_df.csv", "filter_df.csv", "entire_graph.json", "filter_graph.json", "group_df.csv"]
            for f in files:
                if not os.path.exists(dataset_dir + '/' + f):
                    open(os.path.join(dataset_dir, f), 'w').close()

In [197]:
def create_n_runs(incoming_dataset, n, filterBy, filterPerc):
    for i in range(0, n):
        print("Reading dataset: ", incoming_dataset)
        state = read_gf(incoming_dataset)
        dataset = incoming_dataset + '-random-' + str(i)
        create_dot_callflow_folder(dataset)
        state = setvalues(state)
        write_gf(state, dataset, 'entire')
        print("Writing to entire dataset: ", dataset)
        state = ffilter(state, filterBy, filterPerc) 
        print("Writing to filter dataset: ", dataset)
        write_gf(state, dataset, 'filter')
        df_filepath = dirname + '/' + dataset + '/' +'group_df.csv'
        state.group_df.to_csv(df_filepath)

In [198]:
create_n_runs('calc-pi', 10, 'Inclusive', 1)

 CallFlow:   [callfow.py] Action: filter By: "Inclusive"
 CallFlow:   [callfow.py] Action: filter Perc: 1
 CallFlow:  Filtering the graph.
 CallFlow:  [Filter] Removed 112 rows. (time=0.03163719177246094)
 CallFlow:  Squashing the graph.
 CallFlow:  [Squash] 24 rows in dataframe (time=0.0391693115234375)


Reading dataset:  calc-pi
0    0.78
1    0.30
2    0.85
3    0.38
4    0.06
     ... 
63   0.85
64   0.75
65   0.12
66   0.97
67   0.75
Length: 68, dtype: float64
writing file for entire format
DFS on the graph
Root = <program root> [2]
Node = main[Module = cpi:4] time (inc) = 405930.3295490888 time = 0.0
- Node = 62:MPI_Finalize[Module = libmonitor.so.0.0.0:27] time (inc) = 554536.7808823532 time = 0.0
- - Node = PMPI_Finalize[Module = libmpi.so.12.0.5:29] time (inc) = 837081.2485324951 time = 0.0
- - - Node = 294:MPID_Finalize[Module = libmpi.so.12.0.5:31] time (inc) = 643504.6381713771 time = 0.0
- - - - Node = 162:MPIDI_CH3_Finalize[Module = libmpi.so.12.0.5:33] time (inc) = 556704.7859450717 time = 0.0
- - - - - Node = 230:psm_dofinalize[Module = libmpi.so.12.0.5:35] time (inc) = 408464.74827480054 time = 0.0
- - - - - - Node = 36:<unknown procedure>[Module = libpsm_infinipath.so.1.14:37] time (inc) = 523686.9556673034 time = 0.0
- - - - - - - Node = <unknown procedure>[Module = l

 CallFlow:   [callfow.py] Action: filter By: "Inclusive"
 CallFlow:   [callfow.py] Action: filter Perc: 1
 CallFlow:  Filtering the graph.
 CallFlow:  [Filter] Removed 113 rows. (time=0.028240203857421875)
 CallFlow:  Squashing the graph.



- - - Node = 294:MPID_Finalize[Module = libmpi.so.12.0.5:31] time (inc) = 338099.85391804593 time = 0.0
- - - - Node = 162:MPIDI_CH3_Finalize[Module = libmpi.so.12.0.5:33] time (inc) = 648089.0698771112 time = 0.0
Number of nodes in graph 6
Dataframe Information
Size: (24, 19)
Number of nodes in dataframe:  6
Nodes: dict_keys([('162:MPIDI_CH3_Finalize', 33), ('294:MPID_Finalize', 31), ('62:MPI_Finalize', 27), ('<program root>', 2), ('PMPI_Finalize', 29), ('main', 4)])
File path: /home/vidi/Work/llnl/CallFlow/.callflow/calc-pi-random-0/filter_graph.json
Reading dataset:  calc-pi
0    0.40
1    0.98
2    0.69
3    0.97
4    0.80
     ... 
63   0.01
64   0.76
65   0.87
66   0.65
67   0.73
Length: 68, dtype: float64
writing file for entire format
DFS on the graph
Root = <program root> [2]
Node = main[Module = cpi:4] time (inc) = 655702.9943622716 time = 0.0
- Node = 62:MPI_Finalize[Module = libmonitor.so.0.0.0:27] time (inc) = 289412.1193929146 time = 0.0
- - Node = PMPI_Finalize[Module =

 CallFlow:  [Squash] 23 rows in dataframe (time=0.037720441818237305)
 CallFlow:   [callfow.py] Action: filter By: "Inclusive"
 CallFlow:   [callfow.py] Action: filter Perc: 1
 CallFlow:  Filtering the graph.
 CallFlow:  [Filter] Removed 113 rows. (time=0.02837085723876953)
 CallFlow:  Squashing the graph.


...


['162:MPIDI_CH3_Finalize', '294:MPID_Finalize', '62:MPI_Finalize', '<program root>', 'PMPI_Finalize', 'main'] {33, 2, 4, 27, 29, 31}
Writing to filter dataset:  calc-pi-random-7
writing file for filter format
DFS on the graph
Root = <program root> [2]
Node = main[Module = cpi:4] time (inc) = 540416.7894972693 time = 0.0
- Node = 62:MPI_Finalize[Module = libmonitor.so.0.0.0:27] time (inc) = 618725.1927282595 time = 0.0
- - Node = PMPI_Finalize[Module = libmpi.so.12.0.5:29] time (inc) = 381349.6461981436 time = 0.0
- - - Node = 294:MPID_Finalize[Module = libmpi.so.12.0.5:31] time (inc) = 433128.386608309 time = 0.0
- - - - Node = 162:MPIDI_CH3_Finalize[Module = libmpi.so.12.0.5:33] time (inc) = 573835.7222264207 time = 0.0
Number of nodes in graph 6
Dataframe Information
Size: (24, 19)
Number of nodes in dataframe:  6
Nodes: dict_keys([('162:MPIDI_CH3_Finalize', 33), ('294:MPID_Finalize', 31), ('62:MPI_Finalize', 27), ('<program root>', 2), ('PMPI_Finalize', 29), ('main', 4)])
File path

 CallFlow:  [Filter] Removed 112 rows. (time=0.0296785831451416)
 CallFlow:  Squashing the graph.
 CallFlow:  [Squash] 24 rows in dataframe (time=0.03961348533630371)
 CallFlow:   [callfow.py] Action: filter By: "Inclusive"
 CallFlow:   [callfow.py] Action: filter Perc: 1
 CallFlow:  Filtering the graph.


['162:MPIDI_CH3_Finalize', '294:MPID_Finalize', '62:MPI_Finalize', '<program root>', 'PMPI_Finalize', 'main'] {33, 2, 4, 27, 29, 31}
Writing to filter dataset:  calc-pi-random-8
writing file for filter format
DFS on the graph
Root = <program root> [2]
Node = main[Module = cpi:4] time (inc) = 425616.020572313 time = 0.0
- Node = 62:MPI_Finalize[Module = libmonitor.so.0.0.0:27] time (inc) = 445060.0998603711 time = 0.0
- - Node = PMPI_Finalize[Module = libmpi.so.12.0.5:29] time (inc) = 626984.0373551123 time = 0.0
- - - Node = 294:MPID_Finalize[Module = libmpi.so.12.0.5:31] time (inc) = 455292.68473813403 time = 0.0
- - - - Node = 162:MPIDI_CH3_Finalize[Module = libmpi.so.12.0.5:33] time (inc) = 752841.6797633409 time = 0.0
Number of nodes in graph 6
Dataframe Information
Size: (24, 19)
Number of nodes in dataframe:  6
Nodes: dict_keys([('162:MPIDI_CH3_Finalize', 33), ('294:MPID_Finalize', 31), ('62:MPI_Finalize', 27), ('<program root>', 2), ('PMPI_Finalize', 29), ('main', 4)])
File path

 CallFlow:  [Filter] Removed 112 rows. (time=0.04357719421386719)
 CallFlow:  Squashing the graph.
 CallFlow:  [Squash] 24 rows in dataframe (time=0.04484868049621582)


                               node  rank  time (inc)  time  nid  rank.1  \
node           rank                                                        
<program root> 0     <program root>     0  855,479.23  0.00    2       0   
               1     <program root>     1  840,756.22  0.00    2       1   
               2     <program root>     2  610,233.94  0.00    2       2   
               3     <program root>     3  369,564.66  0.00    2       3   

                               file  line               module  \
node           rank                                              
<program root> 0     <unknown file>     0  libmonitor.so.0.0.0   
               1     <unknown file>     0  libmonitor.so.0.0.0   
               2     <unknown file>     0  libmonitor.so.0.0.0   
               3     <unknown file>     0  libmonitor.so.0.0.0   

                               name          node.1 type  n_index  mod_index  \
node           rank                                               

In [148]:
dataset = 'calc-pi-random-0'
state = read_gf(dataset)
print(state.df)

                      node  rank                  node.1  rank.1  time (inc)  \
0           <program root>     0          <program root>       0  246,611.89   
1           <program root>     1          <program root>       1  902,982.46   
2           <program root>     3          <program root>       3  609,100.32   
3                     main     0                    main       0  823,426.11   
4                     main     1                    main       1  670,229.85   
..                     ...   ...                     ...     ...         ...   
18       294:MPID_Finalize     3       294:MPID_Finalize       3  589,598.43   
19  162:MPIDI_CH3_Finalize     0  162:MPIDI_CH3_Finalize       0  319,197.57   
20  162:MPIDI_CH3_Finalize     1  162:MPIDI_CH3_Finalize       1  819,456.53   
21  162:MPIDI_CH3_Finalize     2  162:MPIDI_CH3_Finalize       2  212,572.48   
22  162:MPIDI_CH3_Finalize     3  162:MPIDI_CH3_Finalize       3  795,559.32   

    time  nid  rank.1.1                